# Accuracy Using a Classifier

In [0]:
%matplotlib inline
from fastai.vision import *
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import os

## Data and Imports

In [0]:
colab = True
if colab:
  from google.colab import drive
  drive.mount('/content/drive', force_remount = True)
  %cp "/content/drive/My Drive/autoencoder-training/baseline-MNIST.pt" .
  %cp "/content/drive/My Drive/autoencoder-training/featureLoss-MNIST.pt" .
  %cp "/content/drive/My Drive/autoencoder-training/pixelShuffle-MNIST.pt" .
  %cp "/content/drive/My Drive/autoencoder-training/preTrained-MNIST.pt" .
  %cp "/content/drive/My Drive/autoencoder-training/preTrained_nonweights-MNIST.pt" .
  %cp "/content/drive/My Drive/autoencoder-training/progResizing-MNIST.pt" .

  %cp "/content/drive/My Drive/autoencoder-training/baseline_model.py" .
  %cp "/content/drive/My Drive/autoencoder-training/model_layers.py" .
  %cp "/content/drive/My Drive/autoencoder-training/featureLoss_function.py" .
  %cp "/content/drive/My Drive/autoencoder-training/pixelShuffle_model.py" .
  %cp "/content/drive/My Drive/autoencoder-training/pretrainedModel.py" .
  %cp "/content/drive/My Drive/autoencoder-training/progressiveResizing_model.py" .
  image_path = "data"
  import baseline_model
  import model_layers
  import featureLoss_function
  import pixelShuffle_model
  import pretrainedModel
  import progressiveResizing_model
else: 
  os.chdir("../")
  image_path = os.getcwd() + "/data"

In [0]:
np.random.seed(3333)
torch.manual_seed(3333)

size = 32
batchsize = 128

path = untar_data(URLs.MNIST)
data = ImageDataBunch.from_folder(path, size=32,ds_tfms=get_transforms(p_affine=0), bs = 128
                                 ).normalize(imagenet_stats)

## Training Classifier

In [0]:
learn = cnn_learner(data, models.resnet34, metrics=error_rate)

In [0]:
learn.fit_one_cycle(1)

In [0]:
learn.lr_find()

In [0]:
learn.recorder.plot(suggestion = True)

In [0]:
learn.fit_one_cycle(10, max_lr = )

In [0]:
learn.unfreeze()
learn.fit_one_cycle(10, max_lr = slice(, ))

In [0]:
torch.save(learn.model, "baselineClassification.pt")
torch.save(learn.model, "/content/drive/My Drive/autoencoder-training/baselineClassificationMNIST.pt")

## Classifier Accuracy

### Baseline

In [0]:
class baseline(nn.Module):
    def __init__(self):
        super(baseline, self).__init__()

        self.autoencoder = torch.load("baseline-MNIST.pt")
        self.classifier = torch.load("baselineClassification.pt")
        self.mean = torch.tensor(imagenet_stats[0]).cuda()
        self.std = torch.tensor(imagenet_stats[1]).cuda()

    def forward(self, x):
        x = torch.clamp(self.autoencoder(x), min=0, max=1)
        x = (x - self.mean[...,None,None])/self.std[...,None,None]
        x = self.classifier(x)
        return x

In [0]:
baseline_evaluation = baseline()
learn = Learner(data, baseline_evaluation, metrics = [error_rate, accuracy])

In [0]:
learn.validate(learn.data.valid_dl)

### FeatureLoss

In [0]:
class featureloss_mod(nn.Module):
    def __init__(self):
        super(featureloss_mod, self).__init__()

        self.autoencoder = torch.load("featureLoss-MNIST.pt")
        self.classifier = torch.load("baselineClassification.pt")
        self.mean = torch.tensor(imagenet_stats[0]).cuda()
        self.std = torch.tensor(imagenet_stats[1]).cuda()

    def forward(self, x):
        x = torch.clamp(self.autoencoder(x), min=0, max=1)
        x = (x - self.mean[...,None,None])/self.std[...,None,None]
        x = self.classifier(x)
        return x

In [0]:
featureloss_evaluation = featureloss_mod()
learn = Learner(data, featureloss_evaluation, metrics = [error_rate, accuracy])

In [0]:
learn.validate(learn.data.valid_dl)

### Pretrained Models

#### Pretrained

In [0]:
class pretrained_weights(nn.Module):
    def __init__(self):
        super(pretrained_weights, self).__init__()

        self.autoencoder = torch.load("preTrained-MNIST.pt")
        self.classifier = torch.load("baselineClassification.pt")
        self.mean = torch.tensor(imagenet_stats[0]).cuda()
        self.std = torch.tensor(imagenet_stats[1]).cuda()

    def forward(self, x):
        x = torch.clamp(self.autoencoder(x), min=0, max=1)
        x = (x - self.mean[...,None,None])/self.std[...,None,None]
        x = self.classifier(x)
        return x

In [0]:
pretrained_weights_eval = pretrained_weights()
learn = Learner(data, pretrained_weights_eval, metrics = [error_rate, accuracy])

In [0]:
learn.validate(learn.data.valid_dl)

#### Resnet

In [0]:
class resnet_model(nn.Module):
    def __init__(self):
        super(resnet_model, self).__init__()

        self.autoencoder = torch.load("preTrained_nonweights-MNIST.pt")
        self.classifier = torch.load("baselineClassification.pt")
        self.mean = torch.tensor(imagenet_stats[0]).cuda()
        self.std = torch.tensor(imagenet_stats[1]).cuda()

    def forward(self, x):
        x = torch.clamp(self.autoencoder(x), min=0, max=1)
        x = (x - self.mean[...,None,None])/self.std[...,None,None]
        x = self.classifier(x)
        return x

In [0]:
resnet_eval = resnet_model()
learn = Learner(data, resnet_eval, metrics = [error_rate, accuracy])

In [0]:
learn.validate(learn.data.valid_dl)

### Pixel Shuffled

In [0]:
class pixelShuffle_model(nn.Module):
    def __init__(self):
        super(pixelShuffle_model, self).__init__()

        self.autoencoder = torch.load("pixelShuffle-MNIST.pt")
        self.classifier = torch.load("baselineClassification.pt")
        self.mean = torch.tensor(imagenet_stats[0]).cuda()
        self.std = torch.tensor(imagenet_stats[1]).cuda()

    def forward(self, x):
        x = torch.clamp(self.autoencoder(x), min=0, max=1)
        x = (x - self.mean[...,None,None])/self.std[...,None,None]
        x = self.classifier(x)
        return x

In [0]:
pixelShuffle_eval = pixelShuffle_model()
learn = Learner(data, pixelShuffle_eval, metrics = [error_rate, accuracy])

In [0]:
learn.validate(learn.data.valid_dl)

### Progressive Resizing

In [0]:
class progResize_model(nn.Module):
    def __init__(self):
        super(progResize_model, self).__init__()

        self.autoencoder = torch.load("progResizing-MNIST.pt")
        self.classifier = torch.load("baselineClassification.pt")
        self.mean = torch.tensor(imagenet_stats[0]).cuda()
        self.std = torch.tensor(imagenet_stats[1]).cuda()

    def forward(self, x):
        x = torch.clamp(self.autoencoder(x), min=0, max=1)
        x = (x - self.mean[...,None,None])/self.std[...,None,None]
        x = self.classifier(x)
        return x

In [0]:
progResize_eval = progResize_model()
learn = Learner(data, progResize_eval, metrics = [error_rate, accuracy])

In [0]:
learn.validate(learn.data.valid_dl)